In [5]:
import gzip, os, sys
import argparse
import pandas as pd
from Bio import SeqIO
from _include import log

In [4]:
ass_f_dir='features/'
ass_s_dir='sequences/'
delim=';'
match_str='16S ribosomal RNA.*'
len_min=500
len_max=2000 
overwrite=False

In [16]:

def add_processed_assembly_ids(
		assembly_df,
		fasta_filename,
		fasta_delim=';',
		overwrite=False
	):
	processed_assembly_ids = set([])
	last_assid = ''
	start_index = 0
	if os.path.isfile(fasta_filename) and not overwrite:
		log('* output '+fasta_filename+' already exists: ', end='')
		log('appending to file...', end='')
		with open(fasta_filename, 'r') as fasta_out:
			n_processed_ids = 0
			for record in SeqIO.parse(fasta_out, 'fasta'):
				last_assid = record.id.strip().split(fasta_delim)[0]
				processed_assembly_ids |= set([last_assid])
				n_processed_ids += 1
		try:
			last_checked_index = assembly_df['#assembly_accession'] == last_assid
			start_index = assembly_df.index[last_checked_index].tolist()[0]                    
		except:
			start_index = 0
		log('OK. Added % 5d sequences.' % (n_processed_ids))
	return (processed_assembly_ids, start_index)

def load_gff_feature_table(filename):
	columns = {
		'assembly': str,
		'genomic_accession':str,
		'start': int,
		'end': int, 
		'strand': str,
		'name': str,
		'attributes': str
	}
	return (
		pd.read_csv(
			filename,
			sep='\t',
			compression='gzip',
			usecols=list(columns.keys()),
			dtype=columns,
			keep_default_na=False,
		)					
	)

def build_file_path(ftp_path, base_path, type):
	suffix = {'features': 'feature_table.txt', 'sequences': 'genomic.fna'}
	return os.path.join(
		base_path,
 		os.path.basename(ftp_path) + '_' + suffix[type] + '.gz',
	)

In [6]:
ass_sub = '../data/archaea_assembly_summary_filter_2023-07-13.txt'

In [40]:
fa_out_name = '../test.fa'

assembly_df = pd.read_csv(
	ass_sub,
	sep='\t',
	usecols=['#assembly_accession', 'ftp_path'],
	dtype={'#assembly_accession': str, 'ftp_path': str}
)

processed_assembly_ids, start_index = add_processed_assembly_ids(assembly_df, fa_out_name)
assembly_df = assembly_df[~assembly_df['#assembly_accession'].isin(processed_assembly_ids)]
assembly_df


,#assembly_accession,ftp_path
0,GCF_009428885.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
1,GCF_009729015.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
2,GCF_003201835.2,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
3,GCF_000213215.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
4,GCF_009729545.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
...,...,...
1455,GCF_001316265.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
1456,GCF_001316285.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
1457,GCF_001748385.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...
1458,GCF_902786095.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/9...


In [36]:
def build_paths(ftp_path, base_paths):
	path_prefix = os.path.basename(ftp_path)
	features_paths = os.path.join(base_paths['features'], path_prefix + '_feature_table.txt.gz')
	sequences_paths = os.path.join(base_paths['sequences'], path_prefix + '_genomic.fna.gz')
	return pd.Series(
		[features_paths, sequences_paths],
		index=['features_path', 'sequences_path']
	)

In [41]:
assembly_df[['features_path', 'sequences_path']] = assembly_df['ftp_path'].apply(
    build_paths,
    base_paths = {
        'features': '../data/'+ass_f_dir,
        'sequences': '../data/'+ass_s_dir,
    }
)
assembly_df

,#assembly_accession,ftp_path,features_path,sequences_path
0,GCF_009428885.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_009428885.1_ASM942888v1_f...,../data/sequences/GCF_009428885.1_ASM942888v1_...
1,GCF_009729015.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_009729015.1_ASM972901v1_f...,../data/sequences/GCF_009729015.1_ASM972901v1_...
2,GCF_003201835.2,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_003201835.2_ASM320183v2_f...,../data/sequences/GCF_003201835.2_ASM320183v2_...
3,GCF_000213215.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_000213215.1_ASM21321v1_fe...,../data/sequences/GCF_000213215.1_ASM21321v1_g...
4,GCF_009729545.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_009729545.1_ASM972954v1_f...,../data/sequences/GCF_009729545.1_ASM972954v1_...
...,...,...,...,...
1455,GCF_001316265.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_001316265.1_ASM131626v1_f...,../data/sequences/GCF_001316265.1_ASM131626v1_...
1456,GCF_001316285.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_001316285.1_ASM131628v1_f...,../data/sequences/GCF_001316285.1_ASM131628v1_...
1457,GCF_001748385.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_001748385.1_ASM174838v1_f...,../data/sequences/GCF_001748385.1_ASM174838v1_...
1458,GCF_902786095.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/9...,../data/features/GCF_902786095.1_Rumen_uncultu...,../data/sequences/GCF_902786095.1_Rumen_uncult...


In [47]:
def file_exists(filename):
	return os.path.exists(filename) and os.path.getsize(filename) > 0

def filter_missing_files(assembly_df):
	assembly_df['features_exist'] = assembly_df['features_path'].apply(file_exists)
	assembly_df['sequences_exist'] = assembly_df['sequences_path'].apply(file_exists)
	return (
		assembly_df
		.query('features_exist == True & sequences_exist == True')
		.drop(columns=['features_exist', 'sequences_exist'])
	)

In [48]:
filter_missing_files(assembly_df)

,#assembly_accession,ftp_path,features_path,sequences_path
0,GCF_009428885.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_009428885.1_ASM942888v1_f...,../data/sequences/GCF_009428885.1_ASM942888v1_...
1,GCF_009729015.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_009729015.1_ASM972901v1_f...,../data/sequences/GCF_009729015.1_ASM972901v1_...
2,GCF_003201835.2,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_003201835.2_ASM320183v2_f...,../data/sequences/GCF_003201835.2_ASM320183v2_...
3,GCF_000213215.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_000213215.1_ASM21321v1_fe...,../data/sequences/GCF_000213215.1_ASM21321v1_g...
4,GCF_009729545.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_009729545.1_ASM972954v1_f...,../data/sequences/GCF_009729545.1_ASM972954v1_...
...,...,...,...,...
1445,GCF_014962245.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_014962245.1_ASM1496224v1_...,../data/sequences/GCF_014962245.1_ASM1496224v1...
1446,GCF_000092185.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_000092185.1_ASM9218v1_fea...,../data/sequences/GCF_000092185.1_ASM9218v1_ge...
1447,GCF_000148385.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_000148385.1_ASM14838v1_fe...,../data/sequences/GCF_000148385.1_ASM14838v1_g...
1449,GCF_001316005.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/0...,../data/features/GCF_001316005.1_ASM131600v1_f...,../data/sequences/GCF_001316005.1_ASM131600v1_...


In [70]:
len_max

2000

In [68]:
# Find all genomic_accession for all 16S rRNA

# Load GFF, find 16S records, filter by length
features_filename = assembly_df.loc[3, 'features_path']

def load_16s_records(features_filename, length_thresholds={'min': 500, 'max': 2000}):
	match_str = '16S ribosomal RNA(?!.*methyl).*'
	df = load_gff_feature_table(features_filename)
	features_df = df[df['name'].str.contains(match_str)].copy()
	features_df['length'] = features_df['end'] - features_df['start'] + 1
	features_df = features_df.query(
		f'length >= {length_thresholds["min"]} & length <= {length_thresholds["max"]}'
	)
	return features_df

features_df = load_16s_records(features_filename=)

,assembly,genomic_accession,start,end,strand,name,attributes,length
3035,GCF_000213215.1,NC_015518.1,1296061,1297560,+,16S ribosomal RNA,,1500


In [61]:
sequences_filename = assembly_df.loc[3, 'sequences_path']
sequences_filename

'../data/sequences/GCF_000213215.1_ASM21321v1_genomic.fna.gz'

In [ ]:
with gzip.open(sequences_filename, 'rt') as sequences_fasta:
	for fasta_record in SeqIO.parse(sequences_fasta, 'fasta'):
		features_16s_df = features_df[features_df['genomic_accession'] == fasta_record.id]
		if not features_16s_df.empty:
			for j in range(len(feature_record_df)):
				if ass_f_sub_sub.iloc[j]['strand'] == '-':
					seq = str(record.seq[start:end].reverse_complement())
				else:
					seq = str(record.seq[start:end])
				fa_out.write('>' + ass_id + delim + record.id + delim + 'loc:' + \
								str(start) + ',' + str(end-1) + delim + 'strand:' + \
								ass_f_sub_sub.iloc[j]['strand'] + delim + 'length:' + \
								str(length) + '\n' + \
								seq + '\n')
